<a href="https://colab.research.google.com/github/rahulnani04/CSC-637--SentimentAnalysis-Twitter/blob/main/sentimitent_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import spacy



In [ ]:
col = ['id','country','Label','Text']
data = pd.read_csv("/content/twitter_training.csv")
data.columns = ['id', 'country', 'Label', 'Text']



In [ ]:
data.head()



,id,country,Label,Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
data.tail()


,id,country,Label,Text
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...
74680,9200,Nvidia,Positive,Just like the windows partition of my Mac is l...


In [ ]:
data.shape


data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       74681 non-null  int64 
 1   country  74681 non-null  object
 2   Label    74681 non-null  object
 3   Text     73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
data['Label'].value_counts()



,count
Label,
Negative,22542
Positive,20831
Neutral,18318
Irrelevant,12990


In [ ]:
print(f"{data['Text'][2]} -> {data['Label'][2]}")


im coming on borderlands and i will murder you all, -> Positive


In [ ]:
data.dropna(inplace=True)


In [ ]:
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)

    return " ".join(filtered_tokens)

In [ ]:
data['Preprocessed Text'] = data['Text'].apply(preprocess)


In [ ]:
data


,id,country,Label,Text,Preprocessed Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,come border kill
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,m get borderland kill
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,m come borderland murder
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,m get borderland murder
...,...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...,realize Windows partition Mac like 6 year Nvid...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,realize Mac window partition 6 year Nvidia dri...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,realize window partition Mac 6 year Nvidia dri...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...,realize window partition Mac like 6 year Nvidi...


In [ ]:
# Encoding target column
le = LabelEncoder()
data['Label'] = le.fit_transform(data['Label'])

In [ ]:
data


,id,country,Label,Text,Preprocessed Text
0,2401,Borderlands,3,I am coming to the borders and I will kill you...,come border kill
1,2401,Borderlands,3,im getting on borderlands and i will kill you ...,m get borderland kill
2,2401,Borderlands,3,im coming on borderlands and i will murder you...,m come borderland murder
3,2401,Borderlands,3,im getting on borderlands 2 and i will murder ...,m get borderland 2 murder
4,2401,Borderlands,3,im getting into borderlands and i can murder y...,m get borderland murder
...,...,...,...,...,...
74676,9200,Nvidia,3,Just realized that the Windows partition of my...,realize Windows partition Mac like 6 year Nvid...
74677,9200,Nvidia,3,Just realized that my Mac window partition is ...,realize Mac window partition 6 year Nvidia dri...
74678,9200,Nvidia,3,Just realized the windows partition of my Mac ...,realize window partition Mac 6 year Nvidia dri...
74679,9200,Nvidia,3,Just realized between the windows partition of...,realize window partition Mac like 6 year Nvidi...


In [ ]:
# Split data into train and test

x_train, x_test, y_train, y_test = train_test_split(data['Preprocessed Text'], data['Label'],
                                                    test_size=0.2, random_state=42, stratify=data['Label'])

In [ ]:
x_test.shape




(14799,)

In [ ]:
x_train.shape


(59196,)

In [ ]:
clf = Pipeline([
    ('vectorizer_tri_grams', TfidfVectorizer()),
    ('naive_bayes', (MultinomialNB()))
])

In [ ]:
clf.fit(x_train, y_train)


Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('naive_bayes', MultinomialNB())])

In [ ]:
y_pred = clf.predict(x_test)


In [ ]:
print(accuracy_score(y_test, y_pred))


0.731671058855328


In [ ]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.94      0.46      0.62      2575
           1       0.65      0.90      0.76      4472
           2       0.83      0.64      0.72      3621
           3       0.72      0.81      0.76      4131

    accuracy                           0.73     14799
   macro avg       0.79      0.70      0.71     14799
weighted avg       0.77      0.73      0.72     14799



In [ ]:
clf = Pipeline([
    ('vectorizer_tri_grams', TfidfVectorizer()),
    ('naive_bayes', (LogisticRegression()))
])

In [ ]:
clf.fit(x_train, y_train)


Pipeline(steps=[('vectorizer_tri_grams', TfidfVectorizer()),
                ('naive_bayes', LogisticRegression())])

In [ ]:
y_pred = clf.predict(x_test)



In [ ]:
print(accuracy_score(y_test, y_pred))


0.7816744374619907


In [ ]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.81      0.68      0.74      2575
           1       0.79      0.84      0.82      4472
           2       0.80      0.73      0.76      3621
           3       0.74      0.82      0.78      4131

    accuracy                           0.78     14799
   macro avg       0.79      0.77      0.78     14799
weighted avg       0.78      0.78      0.78     14799



In [ ]:
test_data = pd.read_csv("/content/twitter_training.csv")
test_data.columns = ['id', 'country', 'Label', 'Text']



In [ ]:
test_data.head()


,id,country,Label,Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [ ]:
test_txt = test_data['Text'][20]
print(f"{test_txt} ===> {test_data['Label'][20]}")


that was the first borderlands session in a long time where i actually enjoyed a really satisfying combat experience. i got some rather good kills ===> Positive
that was the first borderlands session in a long time where i actually enjoyed a really satisfying combat experience. i got some rather good kills ===> Positive


In [ ]:
test_txt_processed = [preprocess(test_txt)]
test_txt_processed

['borderland session long time actually enjoy satisfy combat experience get good kill']

In [ ]:
classes = ['Irrelevant', 'Natural', 'Negative', 'Positive']

print(f"True Label: {test_data['Label'][20]}")
# Predict the class label for the preprocessed text
predicted_class_index = clf.predict(test_txt_processed)[0]
print(f'Predict Label: {classes[predicted_class_index]}')

True Label: Positive
Predict Label: Positive
